In [2]:
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '1')
get_ipython().run_line_magic('aimport', 'data')
get_ipython().run_line_magic('aimport', 'methods')

import warnings
from functools import wraps
import pandas as pd
import numpy as np
import os
import string
from pandas import ExcelWriter
from xlsxwriter.utility import xl_col_to_name
import datetime

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)


### Reading Files

In [3]:
DATADIR = '/Users/yanangao/Desktop/Irvine & Venice Combined Files'
DPDSDomainESPSAPUBlist, DPDSDomainlist,DPDSlist,domainNames,mappingDF = data.mapping(
    os.path.join(
        DATADIR,
        'Email Identifier Mapping Tracker.csv',
    )
)

offers = data.offers(
    os.path.join(
        DATADIR,
        'Offer Sheet by Advertiser.csv'
    )
)

brontoRxmg = data.bronto(
    os.path.join(
        DATADIR,
        'Dash - Master Bronto.xlsx'
    ),
    domainNames
)

iterableRxmg = data.iterable(
    os.path.join(
        DATADIR,
        'Dash - Master Iterable.xlsx'
    ),
    domainNames
)

tailormailRxmg = data.tailoredmail(
    os.path.join(
        DATADIR,
        'Dash - Master Tailored Mail.xlsx'
    ),
    domainNames
)

eaRxmg = data.ea(
    os.path.join(
        DATADIR,
        'Dash - Master EA.xlsx'
    ),
)

revenueRxmg = data.revenue(
    os.path.join(
        DATADIR,
        'Master Revenue.csv'
    ),
    offers,
    domainNames
)

orgRxmg = pd.read_csv(
    os.path.join(
        DATADIR,
        'Master Revenue.csv'
    ),
    encoding = 'latin-1'
)

# Combine ESP: Bronto & Iterable
brontoRxmg = pd.concat([brontoRxmg,
                        iterableRxmg,
                        tailormailRxmg,
                       ]).reset_index()
brontoRxmg['index_b'] = brontoRxmg.index

# get name varialbes
startTimeRxmg = brontoRxmg.date_brt.min().date().strftime("%m.%d.%y")
endTimeRxmg = brontoRxmg.date_brt.max().date().strftime("%m.%d.%y")
fullTimeRxmg = " - ".join([startTimeRxmg,endTimeRxmg])
prefixRxmg = "Irvine & Venice Combined Files"
folder_nameRxmg = "Irvine & Venice Combined Files"

# combine EA and Bronto:
df_newRxmg, dup_fullRxmg, missing_fullRxmg = methods.combine_bronto_ea(brontoRxmg, eaRxmg)


--------bronto file report--------

# of Bronto drops: (4673, 23)
+-------------+
|Check point I|
+-------------+
Passed Bronto column data type checking!

--------Iterable file report--------

# of Iterable drops: (1805, 24)

--------Tailored Mail file report--------

# of Tailored Mail drops: (364, 24)

--------EA file report--------

EA rows before combination (5739, 23)
EA rows after combination (5516, 23)

-------revenue file report---------

+--------------+
|Check point II|
+--------------+
Passed duplicated HitPath ID checking!
# of drops in Revenue report: 6892
+---------------+
|Check point III|
+---------------+
Passed Revenue column data type checking!

-------EA match Bronto report---------

# of duplicated drops in brt: (1189, 48)
# of good drops in brt: (4445, 47)
# of unmatches in brt: (1208, 26)
Pass row number check of brt and merging tables

---------------------------------------



### Error message report

In [4]:
# ================================ Rxmg ERROR REPORT V2 ======================================================= #
# message checking report 
eaDomains = list(eaRxmg.sender_domain.unique())
# brtNewStruc = brontoRxmg[brontoRxmg['date_brt'] >= '2019-12-7']
brtNewStruc = brontoRxmg[(brontoRxmg['Message'].str.count('_')>=11)|(brontoRxmg['drop'].str.contains('.C'))]
# revenueNewStruc = revenueRxmg[revenueRxmg['Date'] >= '2019-12-7']
revenueNewStruc = revenueRxmg[(revenueRxmg['Message'].str.count('_')>=11)|(revenueRxmg['drop'].str.contains('.C'))]
errorBrtRxmg, errorIteRxmg,errorTMRxmg,errorrevRxmg, notInRevRxmg, notInSenderRxmg, dupSenderRxmg, dupRevRxmg = methods.error_reportV2(brtNewStruc,revenueNewStruc,fullTimeRxmg,df_newRxmg, dup_fullRxmg, missing_fullRxmg,DPDSDomainESPSAPUBlist,domainNames,eaDomains)

# save error report
methods.error_report_saver(folder_nameRxmg,prefixRxmg,'(new structure)',fullTimeRxmg,errorBrtRxmg, errorIteRxmg,errorTMRxmg,errorrevRxmg,notInRevRxmg,notInSenderRxmg,dupSenderRxmg, dupRevRxmg,missing_fullRxmg)

# ================================ Rxmg ERROR REPORT V1 ======================================================= #
# message checking report 
eaDomains = list(eaRxmg.sender_domain.unique())
# brtOldStruc = brontoRxmg[brontoRxmg['date_brt'] < '2019-12-7']
# revenueOldStruc = revenueRxmg[revenueRxmg['Date'] < '2019-12-7']
brtOldStruc = brontoRxmg[(brontoRxmg['Message'].str.count('_')<11)&(~brontoRxmg['drop'].str.contains('.C'))]
revenueOldStruc = revenueRxmg[(revenueRxmg['Message'].str.count('_')<11)&(~revenueRxmg['drop'].str.contains('.C'))]
errorBrtRxmg, errorIteRxmg, errorTMRxmg,errorrevRxmg, notInRevRxmg, notInSenderRxmg, dupSenderRxmg, dupRevRxmg = methods.error_report(brtOldStruc,revenueOldStruc,fullTimeRxmg,df_newRxmg, dup_fullRxmg, missing_fullRxmg,DPDSDomainlist,domainNames,eaDomains)

# save error report
methods.error_report_saver(folder_nameRxmg,prefixRxmg,'(old structure)',fullTimeRxmg,errorBrtRxmg, errorIteRxmg,errorTMRxmg,errorrevRxmg,notInRevRxmg,notInSenderRxmg,dupSenderRxmg, dupRevRxmg,missing_fullRxmg)




================ Report ===============

Date range: 01.02.20 - 02.02.20

-------------------------------------

Total Bronto drops: 2349
Total Iterable drops: 889
Total Tailored Mail drops: 120
Total Revenue drops: 3389

-------------------------------------

ESPs non-unique dops: 0
Revenue non-unique dops: 0

-------------------------------------

When matching ESPs with Revenue report:
Drops in ESPs not in Revenue file: 11
Drops in Revenue not in ESPs file: 42
Drops in both ESPs and Revenue files: 3347

-------------------------------------

Error messages in Bronto file: 40
Error messages in Iterable file: 48
Error messages in TM file: 30
Error messages in Revenue file: 110

-------------------------------------

When matching ESPs with EA:
# of one-to-one matches: 4445
# of one-to-many matches: 1189
Drops in ESPs not in EA file: 1208

========== Files are attached ==========


================ Report ===============

Date range: 01.02.20 - 02.02.20

------------------------------